In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
import pandas as pd

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
version = "models/gemini-1.5-flash"
model = genai.GenerativeModel(version, generation_config={"temperature": 1.5,
                                                          "top_p": 0.96,
                                                          "max_output_tokens": 600})

In [ ]:
for m in genai.list_models():
  if m.name == 'models/gemini-1.5-flash':
    print(m)

In [ ]:
from google.colab import files
uploaded = files.upload()
cv = list(uploaded.keys())[0]
cv = genai.upload_file(cv)

uploaded = files.upload()
job = pd.read_csv(list(uploaded.keys())[0], sep=",", quotechar='"')
job_title, company_name, job_description = job["job_title"].values[0], job["company_name"].values[0], job["job_description"].values[0]

In [ ]:
print(cv.name)

myfile = genai.get_file(cv.name)
print(myfile)

In [ ]:
print(job_title, company_name, job_description, sep="\n")

In [ ]:
prompt = f"""This is a canditate's CV in pdf format. Extract the text considering layouts, headings and subheadings.
Then write a professional cover letter of 250-330 words based on this CV and the following details:
- Job Title: {job_title}
- Company: {company_name}
- Job Description: {job_description}

Please ensure the cover letter is tailored to the specific role and company. It should:
1. Clearly state the purpose of the letter and introduce the candidate.
2. Demonstrate genuine interest in the job and company.
3. Complement the CV by offering a little more detail about key experiences, not to simply repeat the CV in paragraph form.
4. Focus on transferable skills if no direct experience exists.
5. Maintain a professional tone, be concise, and thank the reader for their time and consideration.

Structure:
- Introduction: State why the candidate is interested in the role and company.
- Body: Provide 2-3 key examples from the CV that show how the candidate's skills and experience align with the role.
- Closing: Reaffirm the candidate's interest and gratitude for the opportunity.

Be sure to personalize the letter and avoid generic language. The body paragraphs must be equal in size. Your response mustn't be more than 340 (three hundred forty) words!
"""

In [ ]:
response = model.generate_content([cv, prompt])
print(response.text)


In [ ]:
prompt_token_count = model.count_tokens(prompt)
output_token_count = model.count_tokens(response.text)
print(f"Tokens in prompt: {prompt_token_count} \n Estimated tokens in output {output_token_count}")

In [ ]:
for i in range(10):
  response = model.generate_content([cv, prompt])
  output_token_count = model.count_tokens(response.text)
  print(f"Estimated words in output {i+1}: {round(output_token_count.total_tokens*0.7)}")